In [3]:
from app.services.hantu_services.auth_service import AuthService


auth_service = AuthService()
# access token 발급
access_token = auth_service.get_access_token()

access token이 발급되었습니다.
access token이 유효합니다.


In [5]:
from app.services.hantu_services.quotations_service import QuotationsService


quotations_service = QuotationsService(access_token)

In [6]:
# 주식현재가 시세
inquire_price_res = quotations_service.inquire_price("005930")

[MCA00000] 정상처리 되었습니다.


In [7]:
print(
    inquire_price_res["stck_prpr"],  # 주식 현재가
    inquire_price_res["prdy_vrss"],  # 전일대비
    inquire_price_res["prdy_ctrt"],  # 전일대비율
    inquire_price_res[
        "prdy_vrss_sign"
    ],  # 전일 대비 부호 1: 상한, 2: 상승, 3: 보합, 4: 하한, 5: 하락
    inquire_price_res["stck_oprc"],  # 주식 시가
    inquire_price_res["stck_hgpr"],  # 주식 최고가
    inquire_price_res["stck_lwpr"],  # 주식 최저가
    inquire_price_res["stck_mxpr"],  # 주식 상한가
    inquire_price_res["stck_llam"],  # 주식 하한가
    inquire_price_res["stck_sdpr"],  # 주식 기준가
    inquire_price_res[
        "mrkt_warn_cls_code"
    ],  # 시장경고코드 00: 없음, 01: 투자주의, 02: 투자경고, 03: 투자위험
)

55300 -800 -1.43 5 56300 56600 55200 72900 39300 56100 00


In [8]:
from app.services.hantu_services.trading_service import TradingService


trading_service = TradingService(access_token)

In [9]:
# 주식주문(현금)
order_cash_res = trading_service.order_cash("005930", "1", "40000")

In [10]:
# 주식일별주문체결조회
inquire_daily_ccld_res = trading_service.inquire_daily_ccld()
print(inquire_daily_ccld_res)
# inquire_daily_ccld_res[0]["odno"]  # 주문번호

[20310000] 모의투자 조회가 완료되었습니다.                                                 
[{'ord_dt': '20241216', 'ord_gno_brno': '00950', 'odno': '0000084756', 'orgn_odno': '0000000000', 'ord_dvsn_name': '보통가', 'sll_buy_dvsn_cd': '02', 'sll_buy_dvsn_cd_name': '매수', 'pdno': '005930', 'prdt_name': '삼성전자', 'ord_qty': '1', 'ord_unpr': '40000', 'ord_tmd': '151810', 'tot_ccld_qty': '0', 'avg_prvs': '0', 'cncl_yn': 'N', 'tot_ccld_amt': '0', 'loan_dt': '', 'ordr_empno': '', 'ord_dvsn_cd': '00', 'cncl_cfrm_qty': '0', 'rmn_qty': '1', 'rjct_qty': '0', 'ccld_cndt_name': '', 'inqr_ip_addr': '', 'cpbc_ordp_ord_rcit_dvsn_cd': '', 'cpbc_ordp_infm_mthd_dvsn_cd': '', 'infm_tmd': '151810', 'ctac_tlno': '', 'prdt_type_cd': '', 'excg_dvsn_cd': '', 'cpbc_ordp_mtrl_dvsn_cd': '', 'ord_orgno': '', 'rsvn_ord_end_dt': ''}]


In [11]:
def filter_ticker(obj):
    return True if obj["pdno"] == "005930" else False
filtered_list = list(filter(filter_ticker, inquire_daily_ccld_res))

if len(filtered_list) > 0:
    # 주식주문(정정취소)
    order_rvsenclcl_res = trading_service.order_rvsecncl(filtered_list[0]["odno"])
    print(order_rvsenclcl_res["msg_cd"], order_rvsenclcl_res["msg1"])
else:
    print("주문번호가 존재하지 않습니다.")

{'rt_cd': '0', 'msg_cd': '40630000', 'msg1': '모의투자 취소주문이 완료 되었습니다.', 'output': {'KRX_FWDG_ORD_ORGNO': '00950', 'ODNO': '84778', 'ORD_TMD': '151820'}}
40630000 모의투자 취소주문이 완료 되었습니다.
